# Data Download and Exploration

This code means that the notebook will re-import your source code in `src` when it is edited (the default is not to re-import, because most modules are assumed not to change over time).  It's a good idea to include it in any exploratory notebook that uses `src` code

In [1]:
%load_ext autoreload
%autoreload 2

This snippet allows the notebook to import from the `src` module.  The directory structure looks like:

```
├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering)
│   │                     followed by the topic of the notebook, e.g.
│   │                     01_data_collection_exploration.ipynb
│   └── exploratory    <- Raw, flow-of-consciousness, work-in-progress notebooks
│   └── report         <- Final summary notebook(s)
│
├── src                <- Source code for use in this project
│   ├── data           <- Scripts to download and query data
│   │   ├── sql        <- SQL scripts. Naming convention is a number (for ordering)
│   │   │                 followed by the topic of the script, e.g.
│   │   │                 03_create_pums_2017_table.sql
│   │   ├── data_collection.py
│   │   └── sql_utils.py
```

So we need to go up two "pardir"s (parent directories) to import the `src` code from this notebook.  You'll want to include this code at the top of any notebook that uses the `src` code.

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

The code to download all of the data and load it into a SQL database is in the `data` module within the `src` module.  You'll only need to run `download_data_and_load_into_sql` one time for the duration of the project.

In [3]:
from src.data import data_collection

This line may take as long as 10-20 minutes depending on your network connection and computer specs

In [4]:
#data_collection.download_data_and_load_into_sql()

Now it's time to explore the data!

In [19]:
import psycopg2
import pandas as pd
pd.set_option('max_colwidth', 80)

In [20]:
DBNAME = "opportunity_youth"

In [51]:
conn = psycopg2.connect(dbname=DBNAME)

In [114]:
df =pd.read_sql("""
SELECT DISTINCT A.serialno, A.puma, A.st, A.agep, A.sch, A.schl, A.fesrp, A.rac1p,
 CASE WHEN A.schl ='1' AND A.fesrp='0' THEN 'Opportunity Youth'
      WHEN A.schl ='1' AND A.fesrp='1' AND A.schl < '16' THEN 'Working without Diploma'
      ELSE 'Not Opportunity Youth' 
      END AS youthtype,
CASE WHEN A.agep BETWEEN 16 AND 18 THEN '16-18'
     WHEN A.agep BETWEEN 19 AND 21 THEN '19-21'
     WHEN A.agep BETWEEN 22 AND 24 THEN '22-24'
     END AS age
FROM (SELECT DISTINCT serialno, puma, st, MAX(agep) OVER (PARTITION BY serialno) AS agep, MAX(sch) OVER (PARTITION BY serialno) AS sch, 
MAX(schl) OVER (PARTITION BY serialno) AS schl, rac1p, fesrp FROM pums_2017 
WHERE st= '53' 
AND (agep >=16 AND agep <=24)) AS A 
;
""", conn)

df

,serialno,puma,st,agep,sch,schl,fesrp,rac1p,youthtype,age
0,2014000139682,11802,53,16.0,2,13,0,1,Not Opportunity Youth,16-18
1,2016000039114,11104,53,18.0,2,20,1,9,Not Opportunity Youth,16-18
2,2014000471273,11103,53,21.0,2,18,0,1,Not Opportunity Youth,19-21
3,2014000252115,11103,53,21.0,1,14,0,1,Not Opportunity Youth,19-21
4,2014000505909,11603,53,20.0,3,19,0,6,Not Opportunity Youth,19-21
...,...,...,...,...,...,...,...,...,...,...
30632,2017001192720,11609,53,22.0,1,21,0,6,Not Opportunity Youth,22-24
30633,2013000703867,11104,53,24.0,1,21,0,1,Not Opportunity Youth,22-24
30634,2013000768774,11801,53,18.0,2,16,0,1,Not Opportunity Youth,16-18
30635,2015001061814,10100,53,17.0,2,13,0,1,Not Opportunity Youth,16-18


In [137]:
pd.read_sql("""

select *

from 
(
SELECT
right(serialno,9)as newid, serialno, agep, schl, rac1p, fesrp, count (right(serialno, 9)) over(partition by serialno) as countid
FROM pums_2017 
WHERE st='53' and puma between '11601' and '11615'
AND agep between 16 and 24  AND sch='1' AND fesrp ='0'

) as a 

where a.countid >3

order by a.serialno

;
""", conn)
#SUM (A.countid * A.pwgtp) AS TOTALNUMBER
#FROM

,newid,serialno,agep,schl,rac1p,fesrp,countid
0,000550228,2013000550228,19.0,19,6,0,4
1,000550228,2013000550228,21.0,16,1,0,4
2,000550228,2013000550228,21.0,17,1,0,4
3,000550228,2013000550228,21.0,19,1,0,4
4,000594659,2013000594659,16.0,14,6,0,5
...,...,...,...,...,...,...,...
68,000130388,2017000130388,24.0,21,6,0,4
69,001081547,2017001081547,21.0,14,1,0,4
70,001081547,2017001081547,21.0,16,1,0,4
71,001081547,2017001081547,19.0,15,1,0,4


In [144]:
df =pd.read_sql("""
SELECT 
SUM(A.countid * A.pwgtp) AS total
FROM 
(SELECT DISTINCT A.serialno, A.pwgtp, count(A.serialno) as countid

FROM (SELECT DISTINCT serialno, puma, st, 
MAX(agep) OVER (PARTITION BY serialno) as agep, 
MAX(sch) OVER (PARTITION BY serialno) AS sch, 
MAX(schl) OVER (PARTITION BY serialno) AS schl, 
MAX(rac1p) OVER (PARTITION BY serialno) AS race, 
MAX(fesrp) OVER(PARTITION BY serialno) AS workstatus, 
MAX(sex) OVER (PARTITION BY serialno) AS sex,
pwgtp FROM pums_2017 WHERE st= '53' 
AND (agep >=16 AND agep <=24) AND (puma BETWEEN '1161' AND '11615'  OR puma='11604')
AND LEFT(serialno, 4) >='2017' AND sch='1' and fesrp ='0') AS A 

GROUP BY A.serialno, A.pwgtp
) AS A 
""", conn)

df
 
#serialno, st, agep, sch, schl, fesrp, C.blklondd, C.blklatdd 
#sch = '1' AND fesrp ='0' AND AND LEFT(serialno, 4)='2017'
#A.puma, A.st, A.agep, A.sch, A.schl, A.fesrp, A.rac1p,
#C.cty, C.ctyname, D.puma_name,
#  CASE WHEN A.schl ='1' AND A.fesrp='0' THEN 'Opportunity Youth'
#       WHEN A.schl ='1' AND A.fesrp='1' AND A.schl < '16' THEN 'Working without Diploma'
#       ELSE 'Not Opportunity Youth' 
#       END AS youthtype,
# CASE WHEN A.agep BETWEEN 16 AND 18 THEN '16-18'
#      WHEN A.agep BETWEEN 19 AND 21 THEN '19-21'
#      WHEN A.agep BETWEEN 22 AND 24 THEN '22-24'
#      END AS age
#LEFT JOIN (SELECT DISTINCT * FROM ct_puma_xwalk WHERE statefp='53')AS K ON K.puma5ce =A.puma 
#LEFT JOIN (SELECT DISTINCT trct, cty, ctyname, blklondd, blklatdd from wa_geo_xwalk WHERE cty='53033') AS C ON K.tractce = RIGHT(trct, 6)
#LEFT JOIN (SELECT DISTINCT puma, puma_name FROM puma_names_2010 WHERE state_fips='53') AS D on A.puma=D.puma

,total
0,9056.0


In [139]:
pd.read_sql("""
SELECT
serialno, agep, schl, rac1p, fesrp
FROM pums_2017 
WHERE st='53' and puma between '11601' and '11615'
AND agep between 16 and 24  AND sch='1' AND fesrp ='0'
AND RIGHT(serialno, 9)='000550228'
; """, conn)


,serialno,agep,schl,rac1p,fesrp
0,2013000550228,21.0,19,1,0
1,2013000550228,21.0,17,1,0
2,2013000550228,21.0,16,1,0
3,2013000550228,19.0,19,6,0


Notice the `LIMIT 10` above.  These tables have a large amount of data in them and **your goal is to use SQL to create your main query, not Pandas**.  Pandas can technically do everything that you need to do, but it will be much slower and more inefficient.  Nevertheless, Pandas is still a useful tool for exploring the data and getting a basic sense of what you're looking at.

In [19]:
#df[(df['agep']>= 16) & (df['agep']<=24)]['cow']
#cow =9 (unemployeed)
#division = 9 (pacific)
#region =4 (west)
#st =53 (Washington)
#sch = 1(school enrollment: has not attended in the last 3 months) page 42
#schl: school level page 42
#esr page 59 employee status recode 
#fesrp= 0 page 121 employee status 0=no
#rac1p race recoded page 103
# weights page 126

#df_set=df[['serialno', 'puma','division', 'region', 'st', 'agep', 'esr', 'nwab']]

6    3
Name: cow, dtype: object

In [22]:
pd.read_sql("""SELECT DISTINCT puma, puma_name
FROM puma_names_2010  AS A
WHERE state_name ='Washington' 
ORDER BY puma, puma_name 
;""", conn)


,puma,puma_name
0,10100,Whatcom County--Bellingham City ...
1,10200,"Skagit, Island & San Juan Counties ..."
2,10300,Chelan & Douglas Counties ...
3,10400,"Stevens, Okanogan, Pend Oreille & Ferry Counties ..."
4,10501,Spokane County (North Central)--Spokane City (North) ...
5,10502,Spokane County (South Central)--Spokane City (South) ...
6,10503,Spokane County (East Central)--Greater Spokane Valley City ...
7,10504,Spokane County (Outer)--Cheney City ...
8,10600,"Whitman, Asotin, Adams, Lincoln, Columbia & Garfield Counties ..."
9,10701,"Benton & Franklin Counties--Pasco, Richland (North) & West Richland Cities ..."


In [23]:
pd.read_sql("""
SELECT DISTINCT trct, cty, ctyname, blklondd, blklatdd from wa_geo_xwalk AS A 


WHERE st='53' AND ctyname like 'King%'

;""", conn)

#(blklondd > -122.530396 AND blklondd < -120.084384) AND (blklatdd > 47.072060 AND blklatdd < 47.788447) 

,trct,cty,ctyname,blklondd,blklatdd
0,53033000100,53033,"King County, WA ...",-122.295678,47.727200
1,53033000100,53033,"King County, WA ...",-122.295498,47.725603
2,53033000100,53033,"King County, WA ...",-122.295482,47.720286
3,53033000100,53033,"King County, WA ...",-122.295476,47.723796
4,53033000100,53033,"King County, WA ...",-122.295191,47.731046
...,...,...,...,...,...
35826,53033990100,53033,"King County, WA ...",-122.346140,47.363858
35827,53033990100,53033,"King County, WA ...",-122.343965,47.373841
35828,53033990100,53033,"King County, WA ...",-122.342136,47.376844
35829,53033990100,53033,"King County, WA ...",-122.341212,47.399598


In [12]:
pd.read_sql("SELECT * FROM ct_puma_xwalk WHERE statefp = '53';", conn)

,statefp,countyfp,tractce,puma5ce
0,53,001,950100,10600
1,53,001,950200,10600
2,53,001,950300,10600
3,53,001,950400,10600
4,53,001,950500,10600
...,...,...,...,...
1453,53,077,940002,10902
1454,53,077,940003,10902
1455,53,077,940004,10902
1456,53,077,940005,10902


In [11]:
pd.read_sql("""SELECT serialno, puma, st, agep, sch, fesrp FROM pums_2017 WHERE st= '53' \
AND (agep >16 AND agep <=24) AND sch = '1' AND fesrp ='0'

;""", conn) 

,serialno,puma,st,agep,sch,fesrp
0,2013000056099,11613,53,24.0,1,0
1,2013000056304,11802,53,24.0,1,0
2,2013000056516,10200,53,17.0,1,0
3,2013000056674,11503,53,22.0,1,0
4,2013000056831,10702,53,22.0,1,0
...,...,...,...,...,...,...
13606,2013000053871,11000,53,20.0,1,0
13607,2013000054670,10800,53,22.0,1,0
13608,2013000055348,11706,53,22.0,1,0
13609,2013000054935,11505,53,23.0,1,0


Make sure you close the DB connection when you are done using it

In [14]:
conn.close()